# Example of using ChatGPT API for article summarization
Refer to: https://colab.research.google.com/drive/15jh4v_TBPsTyIBhi0Fz46gEkjvhzGaBR?usp=sharing


In [10]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Note: using Google CoLab


In [11]:
# install required packages
!pip install openai
!pip install gradio

In [12]:
# import the packages
import openai
import gradio as gr
import json
from typing import List, Dict, Tuple

client = openai.OpenAI()

# Check if you have set your ChatGPT API successfully
# You should see "Set ChatGPT API sucessfully!!" if nothing goes wrong.
try:
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [{'role':'user','content': "test"}],
            max_tokens=1,
    )
    print("Set ChatGPT API sucessfully!!")
except:
    print("There seems to be something wrong with your ChatGPT API. Please follow our demonstration in the slide to get a correct one.")

Set ChatGPT API sucessfully!!


In [14]:
from openai import OpenAI
client = OpenAI()

def chat_complete(messages, model="gpt-5-mini", limit=256):
    """
    Safe wrapper that:
    1) uses a higher token cap (handles models that spend tokens on reasoning),
    2) retries with a larger cap if we get empty content or length finish,
    3) falls back to gpt-3.5-turbo if needed.
    """
    def _once(m, max_tok):
        return client.chat.completions.create(
            model=m,
            messages=messages,
            max_completion_tokens=max_tok,   # important: not max_tokens
            # omit temperature for models that only accept default
        )

    # 1st try
    resp = _once(model, limit)
    content = (resp.choices[0].message.content or "").strip()
    finish = resp.choices[0].finish_reason

    if content:
        return content

    # 2nd try: give the model more room if it hit length or returned empty
    if finish == "length" or not content:
        resp2 = _once(model, max(2 * limit, 256))
        content2 = (resp2.choices[0].message.content or "").strip()
        if content2:
            return content2

    # 3rd try: fallback to a stable baseline that doesn’t use reasoning tokens
    fallback_model = "gpt-3.5-turbo" if model != "gpt-3.5-turbo" else model
    resp3 = _once(fallback_model, max(2 * limit, 256))
    content3 = (resp3.choices[0].message.content or "").strip()
    return content3 if content3 else "No output returned (even after retries)."

SYSTEM_INSTRUCTIONS = (
    "You are a precise writing corrector. "
    "Fix spelling, grammar, punctuation, and casing only. "
    "Preserve meaning. Do not add words or explanations. "
    "Output only the corrected text."
)

def revise_text(text: str, model: str = "gpt-5-mini"):
    if not text or not text.strip():
        return "Please enter text to revise."
    messages = [
        {"role": "system", "content": SYSTEM_INSTRUCTIONS},
        {"role": "user", "content": text},
    ]
    return chat_complete(messages, model=model, limit=256)

import gradio as gr

TEST_CASES = {
    "Case 1: ths is a nic dy!": "ths is a nic dy!",
    "Case 2: hw a u?": "hw a u?",
    "Case 3: Ca yo modiy tis sente?": "Ca yo modiy tis sente?",
}

with gr.Blocks() as demo:
    gr.Markdown("# Part4: Revision (ChatGPT API)")
    with gr.Row():
        model_drop = gr.Dropdown(
            choices=["gpt-5-mini", "gpt-5", "gpt-3.5-turbo"],
            value="gpt-5-mini",
            label="Model"
        )
        case = gr.Dropdown(
            choices=list(TEST_CASES.keys()),
            value="Case 2: hw a u?",
            label="Load assignment test case"
        )

    input_box = gr.Textbox(label="Text to revise", value=TEST_CASES["Case 2: hw a u?"], lines=3)
    output_box = gr.Textbox(label="Corrected text", lines=3, interactive=False)

    def load_case(selected):
        return TEST_CASES[selected]

    gr.Button("Load Selected Case").click(load_case, inputs=case, outputs=input_box)
    gr.Button("Correct").click(lambda txt, m: revise_text(txt, m), inputs=[input_box, model_drop], outputs=output_box)

demo.launch(debug=True)


/tmp/ipython-input-1163845991.py:78: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)  # shows all turns


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b701d064713d12d696.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b701d064713d12d696.gradio.live
